## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

#from hparams import create_hparams
from train import Hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence, text_to_phonemes_to_sequence, _clean_text
from denoiser import Denoiser
from phonemizer import phonemize

In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')

#### Setup hparams

In [3]:
hparams = Hparams()


#### Load model from checkpoint

In [4]:
checkpoint_path = "checkpoint_672500"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [5]:
waveglow_path = 'waveglow_6000'#'waveglow_256channels.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)
#waveglow.

D:\FinalProj\tacotron2\stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


#### Prepare text input

In [35]:
text = "Machine learning is awesome part of information technologies"
text = _clean_text(text, ['english_cleaners'])
text = phonemize(text, language='en-us', backend='espeak', preserve_punctuation=True, njobs=8)
sequence = np.array(text_to_phonemes_to_sequence(text))[None, :]

#sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()
voice = [0]
voice = np.array(voice)
voice = torch.autograd.Variable(torch.from_numpy(voice)).cuda().long()
print(text)

məʃiːn lɜːnɪŋ ɪz ɔːsʌm pɑːɹt ʌv ɪnfɚmeɪʃən tɛknɑːlədʒiz 


#### Decode text input and plot results

In [36]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence, voice)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))
plt.savefig('foo1.png', bbox_inches='tight')

#### Synthesize audio from spectrogram using WaveGlow

In [37]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [38]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 